<a href="https://colab.research.google.com/github/manmeet3/Deep_Learning/blob/master/homework_3_graded/Amazon_Data_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import numpy as np
import pandas as pd
import zipfile
import cv2

from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [0]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data_dir = "/content/drive/My Drive/dl_dataset/kaggle/"

In [0]:
os.environ['KAGGLE_CONFIG_DIR'] = data_dir

In [6]:
%cd /content/drive/My Drive/dl_dataset/kaggle/
!ls

/content/drive/My Drive/dl_dataset/kaggle
kaggle.json  planet  planets-dataset.zip  test-jpg-additional


In [0]:
!kaggle datasets download -d nikitarom/planets-dataset

100% 1.50G/1.50G [00:15<00:00, 74.2MB/s]
100% 1.50G/1.50G [00:15<00:00, 102MB/s] 


In [0]:
!unzip planets-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: test-jpg-additional/test-jpg-additional/file_55.jpg  
  inflating: test-jpg-additional/test-jpg-additional/file_550.jpg  
  inflating: test-jpg-additional/test-jpg-additional/file_5500.jpg  
  inflating: test-jpg-additional/test-jpg-additional/file_5501.jpg  
  inflating: test-jpg-additional/test-jpg-additional/file_5502.jpg  
  inflating: test-jpg-additional/test-jpg-additional/file_5503.jpg  
  inflating: test-jpg-additional/test-jpg-additional/file_5504.jpg  
  inflating: test-jpg-additional/test-jpg-additional/file_5505.jpg  
  inflating: test-jpg-additional/test-jpg-additional/file_5506.jpg  
  inflating: test-jpg-additional/test-jpg-additional/file_5507.jpg  
  inflating: test-jpg-additional/test-jpg-additional/file_5508.jpg  
  inflating: test-jpg-additional/test-jpg-additional/file_5509.jpg  
  inflating: test-jpg-additional/test-jpg-additional/file_551.jpg  
  inflating: test-jpg-additional/test-jpg-additional/fil

In [0]:
train_data = pd.read_csv(data_dir + 'planet/train_classes.csv')

In [8]:
train_data.tail(4)

,image_name,tags
40475,train_40475,cloudy
40476,train_40476,agriculture clear primary
40477,train_40477,agriculture clear primary road
40478,train_40478,agriculture cultivation partly_cloudy primary


In [0]:
split_tags = lambda l: [item for sublist in l for item in sublist]
labels = list(set(split_tags([l.split(' ') for l in train_data['tags'].values])))

In [0]:
label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

In [11]:
!ls

kaggle.json  planet  planets-dataset.zip  test-jpg-additional


In [16]:
x_train = []
x_test = []
y_train = []

for f, tags in tqdm(train_data.values, miniters=1000):
    img = cv2.imread(data_dir + 'planet/' + 'train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train.append(cv2.resize(img, (32, 32)))
    y_train.append(targets)


  6%|▌         | 2233/40479 [09:54<2:02:36,  5.20it/s]

KeyboardInterrupt: ignored

In [17]:
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float16) / 255.

print(x_train.shape)
print(y_train.shape)

(2233, 32, 32, 3)
(2233, 17)


In [0]:
split = 35000
x_train, x_valid, y_train, y_valid = x_train[:split], x_train[split:], y_train[:split], y_train[split:]


In [19]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(32, 32, 3)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))

model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
              optimizer='adam',
              metrics=['accuracy'])

NameError: ignored

In [20]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=4,
          verbose=1,
          validation_data=(x_valid, y_valid))

NameError: ignored

In [0]:
from sklearn.metrics import fbeta_score

p_valid = model.predict(x_valid, batch_size=128)
print(y_valid)
print(p_valid)
print(fbeta_score(y_valid, np.array(p_valid) > 0.2, beta=2, average='samples'))